In [2]:
import threading
from threading import Thread
import numpy as np
# import cv2
import os
import random
import time
import sys
from queue import Empty,Full,Queue
from PIL import Image

In [3]:
class threadsafe_iter:
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
    def __iter__(self):
        return self
    def __next__(self):
        with self.lock:
            return next(self.it)
def get_path_i(paths_count):
    current_path_id = 0
    while True:
        yield current_path_id
        current_path_id  = (current_path_id + 1) % paths_count

In [3]:
b=iter([1,2])
a=threadsafe_iter(b)

In [4]:
for i in a:
    print(i)

1
2


In [4]:
class InputGen:
    def __init__(self, paths, batch_size):
        self.paths = paths
        self.index = 0
        self.batch_size = batch_size
        self.init_count = 0
        self.lock = threading.Lock() #mutex for input path
        self.yield_lock = threading.Lock() #mutex for generator yielding of batch
        self.path_id_generator = threadsafe_iter(get_path_i(len(self.paths))) 
        self.images = []
    
    def get_samples_count(self):
        """ Returns the total number of images needed to train an epoch """
        return len(self.paths)
    def get_batches_count(self):
        """ Returns the total number of batches needed to train an epoch """
        return int(self.get_samples_count() / self.batch_size)
    def pre_process_input(self, im,lb):
        """ Do your pre-processing here
                Need to be thread-safe function"""
        return im, lb
    def next(self):
        return self.__iter__()
    
    def __iter__(self):
        while True:
          #In the start of each epoch we shuffle the data paths      
          with self.lock: 
            if (self.init_count == 0):
                random.shuffle(self.paths)
                self.images, self.batch_paths = [], []
                self.init_count = 1
      #Iterates through the input paths in a thread-safe manner
          for path_id in self.path_id_generator: 
            img = self.paths[path_id]
            print(img)
            img = cv2.imread(img,1)
#             img, label = self.pre_process_input(img)
            #Concurrent access by multiple threads to the lists below
            with self.yield_lock: 
                if (len(self.images)) < self.batch_size:
                    print(img.shape)
                    self.images.append(img)
                if len(self.images) % self.batch_size == 0:          
                    yield np.float32(self.images)
                self.images = []
          #At the end of an epoch we re-init data-structures
          with self.lock: 
            self.init_count = 0
    def __call__(self):
        return self.__iter__()


In [5]:
class thread_killer(object):
    def __init__(self):
        self.to_kill = False
  
    def __call__(self):
        return self.to_kill
  
    def set_tokill(self,tokill):
        self.to_kill = tokill

In [15]:
def threaded_batches_feeder(tokill, batches_queue, dataset_generator):
    while tokill() == False:
        for batch,batch_images in enumerate(dataset_generator):
            #We fill the queue with new fetched batch until we reach the max       size.
            batches_queue.put((batch, (batch_images)), block=True)
            if tokill() == True:
                return


In [16]:
def threaded_cuda_batches(tokill,cuda_batches_queue,batches_queue):
    
    while tokill() == False:
        batch, (batch_images) = batches_queue.get(block=True)
        batch_images_np = np.transpose(batch_images, (0, 3, 1, 2))
        batch_images = torch.from_numpy(batch_images_np)
        batch_images = Variable(batch_images)
        cuda_batches_queue.put((batch, (batch_images)), block=True)
        if tokill() == True:
            return

In [17]:
num_epoches=1000
preprocess_workers = 4
batches_per_epoch = 8
path = "../dataset/256/"

In [9]:
training_set_list = [path+i for i in os.listdir(path)]

In [10]:
len(training_set_list),training_set_list[0]

(1743042, '../dataset/256/000002b66c9c498e_resized.jpg')

In [24]:
train_batches_queue = Queue(maxsize=12)
cuda_batches_queue = Queue(maxsize=3)

In [25]:
training_set_generator = InputGen(training_set_list,batches_per_epoch)

In [26]:
train_thread_killer = thread_killer()
train_thread_killer.set_tokill(False)

In [ ]:
for _ in range(preprocess_workers):
    t = Thread(target=threaded_batches_feeder,args=(train_thread_killer, train_batches_queue, training_set_generator))
    t.start()

In [22]:
cuda_transfers_thread_killer = thread_killer()
cuda_transfers_thread_killer.set_tokill(False)
cudathread = Thread(target=threaded_cuda_batches,args=(cuda_transfers_thread_killer, cuda_batches_queue, train_batches_queue))
cudathread.start()

hi
0


In [ ]:
  
  #model is some Pytorch CNN model
  model.cuda()
  model.train()
  
  #Training set list suppose to be a list of full-paths for all
  #the training images.
  
  #Our train batches queue can hold at max 12 batches at any given time.
  #Once the queue is filled the queue is locked.
  
  #Our numpy batches cuda transferer queue.
  #Once the queue is filled the queue is locked
  #We set maxsize to 3 due to GPU memory size limitations
  
  
  
  
  #We launch 4 threads to do load && pre-process the input images


